#### Preparing the movie review dataset

1. Get the dataset

In [ ]:
# Install the torchtext library to get the IMDB dataset
!pip install torchtext

In [ ]:
from torchtext.datasets import IMDB

train_dataset = IMDB(split = 'train')
test_dataset = IMDB(split = 'test')

2. Create the datasets

In [ ]:
from torch.utils.data.dataset import random_split
torch.manual_seed(1)
train_dataset, valid_dataset = random_split(list(train_dataset), [2000, 500])

3. Find unique tokens

In [ ]:
import re
from collections import Counter, OrderedDict

def tokenizer(text):
    text = re.sub('<[^]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?::-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ''.join(emoticons).replace('-', '')
    tokenized = text.split()
    return tokenized

token_counts = Counter()
for label, line in train_dataset:
    tokens = tokenizer(line)
    token_counts.update(tokens)

print('Vocab-size:', len(token_counts))

4. Encode each unique token into integers

In [ ]:
from torchtext.vocab import vocab

sorted_by_freq_tuples = sorted(token_counts.items(), key=lambda x: x[1], reverse = True)

ordered_dict = OrderedDict(sorted_by_freq_tuples)
vocab = vocab(ordered_dict)
vocab.insert_token("<pad>", 0)
vocab.insert_token("<unk>", 1)

vocab.set_default(1)

In [ ]:
# Define the function for transformation

text_pipeline = lambda x: [vocab[token] for token in tokenixer(x)]

label_pipeline = lambda x: 1. if x == 'pos' else 0.

In [ ]:
# Wrap the encode and transformation function

def collate_batch(batch):
    label_list, text_list, lengths = [], [], []
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline)